In [ ]:
import tensorflow as tf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tensorflow import keras
import os
import pathlib
import zipfile
import tempfile
import math

In [ ]:
from google.colab import drive
drive.mount ('/drive')

Drive already mounted at /drive; to attempt to forcibly remount, call drive.mount("/drive", force_remount=True).


In [ ]:
train = pd.read_csv('/drive/My Drive/sign_mnist_train.csv')
test = pd.read_csv('/drive/My Drive/sign_mnist_test.csv')

In [ ]:
x_train = train.drop(['pixel1'], axis=1).values
y_train = train['label'].values
x_test = test.drop(['pixel1'], axis=1).values
y_test = test['label'].values
x_train = x_train / 255
x_test = x_test / 255

In [ ]:
x_train = np.array(x_train)
x_test = np.array(x_test)
x_train = x_train.reshape(x_train.shape[0],28,28,1)
x_test = x_test.reshape(x_test.shape[0],28,28,1)

In [ ]:
model = keras.Sequential([
        keras.layers.InputLayer(input_shape=(28, 28, 1)),
        keras.layers.Reshape(target_shape=(28, 28, 1)),
        keras.layers.Conv2D(filters=16, kernel_size=(3,3), activation=tf.nn.relu, padding='same'),
        
        keras.layers.Conv2D(filters=32, kernel_size=(6,6), activation=tf.nn.relu, padding='same'),
        
        keras.layers.Conv2D(filters=64, kernel_size=(12,12), activation=tf.nn.relu, padding='same'),

        keras.layers.MaxPooling2D(pool_size=(2,2)),
        keras.layers.Flatten(),
        keras.layers.Dense(1024, activation='relu'),
        keras.layers.Dropout(0.20),
        keras.layers.Dense(25, activation='softmax')                  
])

In [ ]:
model.compile(optimizer='adam',
          loss='sparse_categorical_crossentropy',
          metrics=['accuracy'])

In [ ]:
model.fit(x_train, y_train, epochs=10,validation_data=(x_test, y_test))

Epoch 1/10
858/858 [==============================] - 8s 8ms/step - loss: 0.4470 - accuracy: 0.8633 - val_loss: 0.4088 - val_accuracy: 0.8900
Epoch 2/10
858/858 [==============================] - 6s 7ms/step - loss: 0.0043 - accuracy: 0.9988 - val_loss: 0.4187 - val_accuracy: 0.9172
Epoch 3/10
858/858 [==============================] - 6s 7ms/step - loss: 0.0205 - accuracy: 0.9936 - val_loss: 0.6241 - val_accuracy: 0.8826
Epoch 4/10
858/858 [==============================] - 6s 7ms/step - loss: 0.0106 - accuracy: 0.9969 - val_loss: 0.3664 - val_accuracy: 0.9137
Epoch 5/10
858/858 [==============================] - 6s 7ms/step - loss: 0.0135 - accuracy: 0.9962 - val_loss: 0.6008 - val_accuracy: 0.8678
Epoch 6/10
858/858 [==============================] - 6s 7ms/step - loss: 0.0062 - accuracy: 0.9982 - val_loss: 0.4250 - val_accuracy: 0.9274
Epoch 7/10
858/858 [==============================] - 6s 7ms/step - loss: 0.0152 - accuracy: 0.9959 - val_loss: 0.7044 - val_accuracy: 0.8999
Epoch 

In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
reshape (Reshape)            (None, 28, 28, 1)         0         
_________________________________________________________________
conv2d (Conv2D)              (None, 28, 28, 16)        160       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 28, 28, 32)        18464     
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 28, 28, 64)        294976    
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 14, 14, 64)        0         
_________________________________________________________________
flatten (Flatten)            (None, 12544)             0         
_________________________________________________________________
dense (Dense)                (None, 1024)              1

In [ ]:
test_loss, test_acc = model.evaluate(x_test,  y_test, verbose=2)
print('\nTest accuracy:', test_acc)

225/225 - 1s - loss: 0.5006 - accuracy: 0.9456

Test accuracy: 0.9456218481063843


In [ ]:
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

INFO:tensorflow:Assets written to: /tmp/tmpg583v9h_/assets


In [ ]:
tflite_models_dir = pathlib.Path("/tmp/mnist_tflite_models/")
tflite_models_dir.mkdir(exist_ok=True, parents=True)

In [ ]:
tflite_model_file = tflite_models_dir/"sign_mnist.tflite"
tflite_model_file.write_bytes(tflite_model)

52744584

In [ ]:
converter.optimizations = [tf.lite.Optimize.DEFAULT]
tflite_quant_model = converter.convert()
tflite_model_quant_file = tflite_models_dir/"sign_mnist_quant_dyn.tflite"
tflite_model_quant_file.write_bytes(tflite_quant_model)

INFO:tensorflow:Assets written to: /tmp/tmpruqu6dnt/assets


INFO:tensorflow:Assets written to: /tmp/tmpruqu6dnt/assets


13194880

In [ ]:
ls -lh {tflite_models_dir}

total 76M
-rw-r--r-- 1 root root 13M Oct 11 04:16 sign_mnist_model_quant_dyn.tflite
-rw-r--r-- 1 root root 13M Oct 11 04:29 sign_mnist_quant_dyn.tflite
-rw-r--r-- 1 root root 51M Oct 11 04:29 sign_mnist.tflite


In [ ]:
interpreter = tf.lite.Interpreter(model_path=str(tflite_model_file))
interpreter.allocate_tensors()

In [ ]:
interpreter_quant = tf.lite.Interpreter(model_path=str(tflite_model_quant_file))
interpreter_quant.allocate_tensors()

In [ ]:
# A helper function to evaluate the TF Lite model using "test" dataset.
def evaluate_model(interpreter):
  input_index = interpreter.get_input_details()[0]["index"]
  output_index = interpreter.get_output_details()[0]["index"]

  # Run predictions on every image in the "test" dataset.
  prediction_digits = []
  for test_image in x_test:
    # Pre-processing: add batch dimension and convert to float32 to match with
    # the model's input data format.
    test_image = np.expand_dims(test_image, axis=0).astype(np.float32)
    interpreter.set_tensor(input_index, test_image)

    # Run inference.
    interpreter.invoke()

    # Post-processing: remove batch dimension and find the digit with highest
    # probability.
    output = interpreter.tensor(output_index)
    digit = np.argmax(output()[0])
    prediction_digits.append(digit)

  # Compare prediction results with ground truth labels to calculate accuracy.
  accurate_count = 0
  for index in range(len(prediction_digits)):
    if prediction_digits[index] == y_test[index]:
      accurate_count += 1
  accuracy = accurate_count * 1.0 / len(prediction_digits)

  return accuracy

In [ ]:
print(evaluate_model(interpreter))

0.9456218627997769


In [ ]:
print(evaluate_model(interpreter_quant))

0.9456218627997769


In [ ]:
def get_gzipped_model_size(file):
  # It returns the size of the gzipped model in bytes.
  import os
  import zipfile

  _, zipped_file = tempfile.mkstemp('.zip')
  with zipfile.ZipFile(zipped_file, 'w', compression=zipfile.ZIP_DEFLATED) as f:
    f.write(file)

  return os.path.getsize(zipped_file)

In [ ]:
print("Size of gzipped baseline Keras model: %.2f bytes" % (get_gzipped_model_size(tflite_model_file)))
print("Size of gzipped dynamically quantized Keras model: %.2f bytes" % (get_gzipped_model_size(tflite_model_quant_file)))

Size of gzipped baseline Keras model: 48938774.00 bytes
Size of gzipped dynamically quantized Keras model: 9197623.00 bytes
